In [2]:
print ("Hello world")

Hello world


In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from datetime import datetime
import os


# Define the list of stimulations, rats, frequencies, files etc.
stim = ['TIS']
rats = ['Trial 240125']
delta_frequencies = [10, 20, 30]
frequency_filenames = [
    '2-and-2-001-kHz-continuousDLC_resnet50_PNS Trial 240125Feb20shuffle1_3500_filtered.csv',
    '2-and-2-002-kHz-continuousDLC_resnet50_PNS Trial 240125Feb20shuffle1_3500_filtered.csv',
    '2-and-2-003-kHz-continuousDLC_resnet50_PNS Trial 240125Feb20shuffle1_3500_filtered.csv' ]

# List to store individual DataFrames
dfs = []

# Iterate over rats and frequencies
for stim_type in stim:
    for rat in rats:
        for freq, freq_filename in zip(delta_frequencies, frequency_filenames):
            
            # Check if the file exists
            if os.path.exists(freq_filename):
                # Read the CSV file
                df = pd.read_csv(freq_filename)

                # Add a new columns with constant values across rows
                df['Stim type'] = stim_type
                df['Animal ID'] = rat
                df['Stim Freq'] = freq

                # Extract time column
                time_column = df['Time']

                # Define body parts and their sub-columns
                body_parts = ['Hip', 'Knee', 'Ankle', 'Heel (ventral)', 'Instep (dorsal)', 'Toe 1 (Proximal Thumb)', 'Toe 2', 'Toe 3 (Middle)', 'Toe 4', 'Toe 5 (Distal Pinky)']
                sub_columns = ['x', 'y', 'likelihood']

                # Create a dictionary to store all body parts data
                body_parts_data = {}

                # Iterate over body parts
                for part in body_parts:
                    part_data = {}
                    for sub_column in sub_columns:
                        column_name = f'{part} {sub_column}'
                        part_data[sub_column] = df[column_name]
                    body_parts_data[part] = part_data
            
                # Append the DataFrame to the list
                dfs.append(df)

            else:
                print(f'File {freq_filename} not found.')

# Concatenate all DataFrames in the list
df = pd.concat(dfs, ignore_index=True)
df = df.drop(df.columns[[31, 32]], axis=1) ## Somehow there's two columns of NaN values so delete it


In [2]:
# Print to check data frame

# Acess a specific column
body_parts_data['Hip']['x']

# Print the first few rows of the data frame
print(df.head())

   Time       Hip x       Hip y  Hip likelihood      Knee x      Knee y  \
0     0  907.364014  323.449097        0.508190  565.053101  597.282776   
1     1  907.364014  323.449097        0.515557  565.239197  597.282776   
2     2  907.364014  323.449097        0.530301  565.239197  597.282776   
3     3  906.955566  323.229004        0.505865  565.053101  596.848633   
4     4  902.333008  316.175140        0.501654  565.017883  596.685608   

   Knee likelihood     Ankle x     Ankle y  Ankle likelihood  ...  \
0         0.979974  568.747742  808.351929          0.973306  ...   
1         0.979890  568.747742  808.351929          0.972856  ...   
2         0.979608  568.747742  809.214294          0.973140  ...   
3         0.979009  568.601379  808.558533          0.972475  ...   
4         0.978754  567.276611  808.558533          0.972355  ...   

   Toe 3 (Middle) likelihood     Toe 4 x     Toe 4 y  Toe 4 likelihood  \
0                   0.975188  262.770935  881.099487        

In [ ]:
# Create new data frame for Linear Mixed Model (LMM)

new_df = df.copy()

# Concatenate x and y coordinates for each body part
for part in body_parts:
    new_df[f'{part} Coordinates'] = list(zip(new_df[f'{part} x'], new_df[f'{part} y']))

# Edit column names for proper model syntax
new_df.columns = [col.replace('(', '').replace(')', '') for col in new_df.columns]
new_df.columns = [col.replace(' ', '_') for col in new_df.columns]

# Make new data frame with deleted columns not used in LMM
new_df = new_df.drop(new_df.columns[1:31], axis=1)


In [11]:
## Linear Mixed Model (LMM)

import statsmodels.api as sm
from sklearn.decomposition import PCA


# Define fixed and random effects
    
# fixed_effects = ['Stim type', 'Stim Freq', 'Hip Coordinates', 'Knee Coordinates','Ankle Coordinates',
#                  '`Heel (ventral) Coordinates`', '`Instep (dorsal) Coordinates`','`Toe 1 (Proximal Thumb) Coordinates`',
#                  'Toe 2 Coordinates', '`Toe 3 (Middle) Coordinates`', 'Toe 4 Coordinates', '`Toe 5 (Distal Pinky) Coordinates`', 
#                  'Time']

fixed_effects = ['Stim_type', 
                 'Stim_Freq', 
                 'C(Time):Stim_type', 
                'C(Time):Stim_Freq', 
                'C(Time):Stim_type:Stim_Freq'
                ]

random_effects = ['Animal ID']

outcome = ['Hip_Coordinates', 'Knee_Coordinates', 'Ankle_Coordinates',
            'Heel_ventral_Coordinates', 'Instep_dorsal_Coordinates', 'Toe_1_Proximal_Thumb_Coordinates',
            'Toe_2_Coordinates', 'Toe_3_Middle_Coordinates', 'Toe_4_Coordinates', 'Toe_5_Distal_Pinky_Coordinates']

# Construct model formula
model_formula = ' + '.join(fixed_effects)
outcome_variable = ' + '.join(outcome)

# Fit the mixed-effects model with the formula
model = sm.MixedLM.from_formula(f'{outcome_variable} ~ {model_formula}', data=new_df, groups=random_effects)
result = model.fit()

# Print model summary
print(result.summary())



ValueError: endog has evaluated to an array with multiple columns that has shape (4674, 39746). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).

In [41]:
# # Print the first few rows of the data frame
# print(df.head())

# Print column labels and their corresponding column numbers
for i, col_label in enumerate(df.columns):
    print(f"Column {i}: {col_label}")

Column 0: Time
Column 1: Hip_x
Column 2: Hip_y
Column 3: Hip_likelihood
Column 4: Knee_x
Column 5: Knee_y
Column 6: Knee_likelihood
Column 7: Ankle_x
Column 8: Ankle_y
Column 9: Ankle_likelihood
Column 10: Heel_ventral_x
Column 11: Heel_ventral_y
Column 12: Heel_ventral_likelihood
Column 13: Instep_dorsal_x
Column 14: Instep_dorsal_y
Column 15: Instep_dorsal_likelihood
Column 16: Toe_1_Proximal_Thumb_x
Column 17: Toe_1_Proximal_Thumb_y
Column 18: Toe_1_Proximal_Thumb_likelihood
Column 19: Toe_2_x
Column 20: Toe_2_y
Column 21: Toe_2_likelihood
Column 22: Toe_3_Middle_x
Column 23: Toe_3_Middle_y
Column 24: Toe_3_Middle_likelihood
Column 25: Toe_4_x
Column 26: Toe_4_y
Column 27: Toe_4_likelihood
Column 28: Toe_5_Distal_Pinky_x
Column 29: Toe_5_Distal_Pinky_y
Column 30: Toe_5_Distal_Pinky_likelihood
Column 31: Stim_type
Column 32: Animal_ID
Column 33: Stim_Freq
Column 34: Hip_Coordinates
Column 35: Knee_Coordinates
Column 36: Ankle_Coordinates
Column 37: Heel_ventral_Coordinates
Column 38:

In [ ]:
from sklearn.decomposition import PCA

# Initialize PCA with desired number of components
pca = PCA(n_components=2)  # Adjust the number of components as needed

# Fit PCA to the data
pca.fit(df)

# Transform the data to the new coordinate system
transformed_data = pca.transform(df)

# Print the transformed data
print(transformed_data)

In [ ]:
import matplotlib.pyplot as plt

# Scree plot
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.show()

#Cumulative explained variance
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Plot')
plt.show()

# Elbow plot
explained_variance = pca.explained_variance_ratio_
differences = np.diff(explained_variance)
plt.plot(range(1, len(differences) + 1), differences, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Change in Explained Variance')
plt.title('Elbow Method')
plt.show()
